In [86]:
import os
import xml.etree.ElementTree as ET
import re

In [81]:
import csv
import numpy as np
path = "data_info.csv"
with open(path,'w') as f:
    csv_write = csv.writer(f)
    csv_head = ["Time","Temperature","Relative Humidity"]
    # csv_head = np.array(csv_head)
    # str.encode(csv_head,'utf-8')
    csv_write.writerow(csv_head)

In [83]:
def write_xml(file_path, xml_path, file_name):

    tree = ET.parse(file_path)
    root = tree.getroot()
    
    data_row = []
    
    # Open the new file in write mode
    with open(file_name + '.txt', 'w') as f:
        for item in root.iter('item'):
            # Extract and write the time
            time = item.find('pubDate').text
            f.write(f'Time: {time}\n')
            data_row.append(time)
    
            # Extract and write the air temperature and relative humidity
            description = item.find('description').text
            air_temp = re.search('Air temperature : (.*) degrees Celsius', description).group(1)
            rel_humidity = re.search('Relative Humidity : (.*) per cent', description).group(1)
            f.write(f'Air Temperature: {air_temp} degrees Celsius\n')
            f.write(f'Relative Humidity: {rel_humidity} per cent\n')
            data_row.append(air_temp)
            data_row.append(rel_humidity)
            print(data_row)
    
            # Extract and write the air temperatures at other places
            f.write('Air Temperatures at Other Places:\n')
            for match in re.finditer('<tr><td><font size="-1">(.*?)</font></td><td width="100" align="right"><font size="-1">(.*?) ;</font></td></tr>', description):
                place_temp = match.group(1) + ' ' + match.group(2)
                f.write(f'{place_temp}\n')
    
            # Extract and write the rainfall recorded in various regions
            f.write('Rainfall Recorded in Various Regions:\n')
            for match in re.finditer('<tr><td>(.*?)</td><td width="100" align="right">(.*?)&nbsp;mm.', description):
                rainfall = match.group(1) + ' ' + match.group(2) + ' mm' 
                f.write(f'{rainfall}\n')
    
    # write the xml file
    with open(xml_path, 'a+') as f:
        csv_write = csv.writer(f)
        csv_write.writerow(data_row)

In [76]:
path = '/Users/lyu/School/SemeterA/Big_Data/project/data'

path_list = os.listdir(path)

path_list.remove('.DS_Store')

path_list.sort()

print(path_list)

['20221001-0001.xml', '20221001-0101.xml', '20221001-0201.xml', '20221001-0301.xml', '20221001-0401.xml', '20221001-0501.xml', '20221001-0601.xml', '20221001-0701.xml', '20221001-0801.xml', '20221001-0901.xml', '20221001-1001.xml', '20221001-1101.xml', '20221001-1201.xml', '20221001-1301.xml', '20221001-1401.xml', '20221001-1501.xml', '20221001-1601.xml', '20221001-1701.xml', '20221001-1801.xml', '20221001-1901.xml', '20221001-2001.xml', '20221001-2101.xml', '20221001-2201.xml', '20221001-2301.xml', '20221002-0001.xml', '20221002-0101.xml', '20221002-0201.xml', '20221002-0301.xml', '20221002-0401.xml', '20221002-0501.xml', '20221002-0601.xml', '20221002-0701.xml', '20221002-0801.xml', '20221002-0901.xml', '20221002-1001.xml', '20221002-1101.xml', '20221002-1201.xml', '20221002-1301.xml', '20221002-1401.xml', '20221002-1501.xml', '20221002-1601.xml', '20221002-1701.xml', '20221002-1801.xml', '20221002-1901.xml', '20221002-2001.xml', '20221002-2101.xml', '20221002-2201.xml', '20221002-23

In [85]:
xml_path = '/Users/lyu/School/SemeterA/Big_Data/project/data_info.csv'

for f in path_list:
    file_path = '/Users/lyu/School/SemeterA/Big_Data/project/data/' + f
    file_name = f[0:13]
    final_file_name = '/Users/lyu/School/SemeterA/Big_Data/project/data_txt/' + file_name
    write_xml(file_path, xml_path, final_file_name)

['Fri, 30 Sep 2022 15:11:00 GMT', '26', '93']
['Fri, 30 Sep 2022 16:02:00 GMT', '26', '93']
['Fri, 30 Sep 2022 17:02:00 GMT', '26', '93']
['Fri, 30 Sep 2022 18:02:00 GMT', '26', '93']
['Fri, 30 Sep 2022 19:02:00 GMT', '26', '94']
['Fri, 30 Sep 2022 20:02:00 GMT', '26', '93']
['Fri, 30 Sep 2022 21:02:00 GMT', '26', '92']
['Fri, 30 Sep 2022 22:02:00 GMT', '27', '91']
['Fri, 30 Sep 2022 23:02:00 GMT', '27', '90']
['Sat, 01 Oct 2022 00:02:00 GMT', '27', '87']
['Sat, 01 Oct 2022 01:02:00 GMT', '28', '87']
['Sat, 01 Oct 2022 02:02:00 GMT', '28', '84']
['Sat, 01 Oct 2022 03:02:00 GMT', '29', '82']
['Sat, 01 Oct 2022 04:02:00 GMT', '29', '78']
['Sat, 01 Oct 2022 05:30:00 GMT', '29', '87']
['Sat, 01 Oct 2022 06:31:00 GMT', '30', '76']
['Sat, 01 Oct 2022 07:26:00 GMT', '29', '81']
['Sat, 01 Oct 2022 08:02:00 GMT', '28', '83']
['Sat, 01 Oct 2022 09:11:00 GMT', '28', '83']
['Sat, 01 Oct 2022 10:02:00 GMT', '28', '83']
['Sat, 01 Oct 2022 11:02:00 GMT', '28', '84']
['Sat, 01 Oct 2022 12:02:00 GMT', 